# Covid-19 Statistics Aachen: Datenaufbereitung

Die rohen Fallzahlen werden zunächst aus dem im ersten Schritt erzeugten Excel-Sheet eingelesen und dann aufbereitet. Anschließend wird eine Reihe von Kennwerten berechnet. Die Ergebnisse werden in eine zweite Excel-Datei exportiert; die zwischengespeicherten Daten werden im folgenden Schritt zur Erstellung von Diagrammen verwendet.

## Benötigte Imports und Initialisierungen

In [4]:
import pandas as pd
import numpy as np

import configparser

# Konfiguration einlesen
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

## Einlesen der Excel-Datei und Anzeige der Daten der letzten 14 Tage
- Datei: Siehe `config.ini`
- Seite: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **D**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **F**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **G**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spaltentypen: Spalte A als Datum interpretieren
- Die ersten beiden Zeilen (Header) überspringen
- Label der Spalten explizit setzen

In [5]:
c19_cases = pd.read_excel(config['Rohdaten']['FileName'], 
                          sheet_name=config['Rohdaten']['SheetName'],
                          usecols='A,C,D,E,F,G', 
                          index_col=0,
                          parse_dates=[0],
                          skiprows=[1],
                          names=['Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ])

c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
Datum,,,,,
2020-03-09,58,20,NaN,NaN,58
2020-03-10,61,21,NaN,6.0,55
2020-03-11,63,22,NaN,NaN,63
2020-03-12,70,25,NaN,NaN,70
2020-03-13,85,34,NaN,27.0,58
2020-03-14,100,40,NaN,28.0,72
2020-03-15,155,75,NaN,NaN,155
2020-03-16,169,85,NaN,33.0,136
2020-03-17,211,100,2.0,NaN,209


## Behandlung fehlender Werte
### Spalte 'Summe Todesfälle'
Am 17.03.2020 wurde das erste Mal in der offiziellen Verlautbarung ein Todesfall erwähnt; für alle Tage vorher wird dieser Spaltenwert zu Null angenommen. Wenn an einem Tag nach dem 17.03. kein Todesfall in der Meldung verzeichnet ist, wird dieser Spaltenwert jeweils fortgeschrieben mit dem letzten offiziell gemeldeten; dadurch kann es zu Sprüngen in der Statistik kommen.
### Spalte 'Summe genesen'
Das erste Mal wurde in der offiziellen Verlautbarung am 10.03.2020 die Zahl der wieder Genesenen erwähnt, bis zum 31.03. erfolgten nur einzelne Meldungen zu sog. "Freitestungen". Erst danach wird die Zahl der Genesenen regelmäßig in den Meldungen der Krisenstäbe erwähnt. Für alle Tage, für die keine Meldung existiert, wird dieser Spaltenwert jeweils fortgeschrieben mit dem letzten offiziell gemeldeten; dadurch kann es zu Sprüngen in der Statistik kommen.
### Hinzufügen von Tagen, für die keine Fallzahlen vorliegen
Ab dem 01.05.2020 sind die Krisenstäbe von Städteregion und der Stadt Aachen dazu übergegangen, neue Fallzahlen nur noch an normalen Wochentagen zu veröffentlichen; ab Juni wurden nur noch 3x pro Woche die Fallzahlen bekannt gegeben. Damit enthalten die aus dem Excelsheet eingelesenen Zeitreihen tageweise Lücken, insbesondere für die Wochenenden. Diese werden in diesem Schritt aufgefüllt, um die Berechnung gleitender Mittelwerte und Summen später zu vereinfachen.

In [6]:
# Zunächst alle Werte für Tage, an denen sie nicht in der Meldung aufgeführt sind, mit dem jeweils letzten bekannten
# Wert fortschreiben. Das ist möglich, weil alle in Frage kommenden Spalten kumulierte oder fortlaufende Werte enthalten. 
c19_cases = c19_cases.fillna(method='ffill')

# Im zweiten Schritt nun alle verbleibende Werte mit 0 füllen. Das trifft für alle Tage zu, bevor ein Wert das 
# erste Mal in einer Meldung enthalten war.
c19_cases = c19_cases.fillna(0)

# Hinzufügen von Tagen, für die keine Fallzahlen vorliegen:
#   Dataframe neu indizieren
#   Fehlende Spaltenwerte auffüllen, indem die jeweils letzten Werte fortgeschrieben werden
#   FIX: Vorher prüfen, ob Index richtig sortiert?
c19_cases = c19_cases.reindex(pd.date_range(c19_cases.index[0], c19_cases.index[-1]), method='bfill')

## Korrektur der Spaltenformate
Die Fallzahlen wurden wegen fehlender Werte teilweise als Fließkommazahlen (NaN erfordert ein Float zu Darstellung) importiert, es handelt sich aber in allen Spalten um Ganzzahlen.

In [7]:
# Explizite Umwandlung aller Spalten (außer Index) nach Integer
c19_cases = c19_cases.astype('int32')
c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
2020-03-09,58,20,0,0,58
2020-03-10,61,21,0,6,55
2020-03-11,63,22,0,6,63
2020-03-12,70,25,0,6,70
2020-03-13,85,34,0,27,58
2020-03-14,100,40,0,28,72
2020-03-15,155,75,0,28,155
2020-03-16,169,85,0,33,136
2020-03-17,211,100,2,33,209
2020-03-18,273,130,4,33,269


## Berechnung zusätzlicher Spalten

- Fälle in der Städteregion (ohne Aachen) berechnen
- Aktive Fälle (für die Städteregion) berechnen
- Prozentuales Wachstum (oder Rückgang) der aktiven Fälle (für die Städteregion) berechnen
- Gleitendes Mittel (7 Tage) des prozentualem Wachstums (oder Rückgangs) der aktiven Fälle 
  (für die Städteregion) berechnen
- Summen der Fälle bezogen auf 100.000 Einwohner berechnen
- Anzahl der Neuinfektionen pro Tag (für Stadt Aachen und Städteregion) berechnen
- Summe der Neuinfektionen der letzten 7 Tage pro 100.000 Einwohner (für Stadt Aachen und Städteregion) berechnen

In [8]:
# Einwohnerzahlen, Stand 31.12.2018
Einwohner_StädteRegion = 555465
Einwohner_StadtAachen = 247380

def change_per_cent(x):
    """Evaluates and returns the change of the active cases per day.

    Für Berechnung der prozentualen Änderung der aktiven Fälle pro Tag per apply():
    Berechnet die Differenz zwischen dem ersten und letzten Wert des als Array übergebenem 'Rolling Window' 
    und dividiert dann durch den ersten Wert.
    """
    if x.size == 1:
        return 0
    else:
        return (x[-1] - x[0]) / x[0] * 100

def diff(x):
    """Für Berechnung von neuen Infektionen / Genesenen / Todesfällen pro Tag per apply():
    Berechnet die Differenz zwischen dem ersten und letzten Wert des als Array übergebenem 'Rolling Window'
    """
    if x.size == 1:
        return 0
    else:
        return (x[-1] - x[0])

# Fälle in der Städteregion ohne Aachen
c19_cases['Summe StädteRegion'] = c19_cases['Summe'] - c19_cases['Summe Aachen']

# Aktive Fälle
c19_cases['Änderung Akute Fälle [%]'] = c19_cases['Akute Fälle'].rolling(2, min_periods=1).apply(change_per_cent, raw=True)
c19_cases['Änderung Akute Fälle (geglättet) [%]'] = c19_cases['Änderung Akute Fälle [%]'].rolling(7).mean()

# Neuinfektionen
c19_cases['Neuinfektionen'] = c19_cases['Summe'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neuinfektionen Aachen'] = c19_cases['Summe Aachen'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
# Neue Todesfälle
c19_cases['Neue Todesfälle'] = c19_cases['Summe Todesfälle'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
# Neue Genesene
c19_cases['Neue Genesene'] = c19_cases['Summe genesen'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')

# Geglättete Werte (über 7 Tage gemittelt)
c19_cases['Neuinfektionen (geglättet)'] = c19_cases['Neuinfektionen'].rolling(7).mean()
c19_cases['Neue Todesfälle (geglättet)'] = c19_cases['Neue Todesfälle'].rolling(7).mean()
c19_cases['Neue Genesene (geglättet)'] = c19_cases['Neue Genesene'].rolling(7).mean()

# Fälle pro 100.000 Einwohner
c19_cases['Summe pro 100000'] = c19_cases['Summe'] / Einwohner_StädteRegion * 100000
c19_cases['Summe Aachen pro 100000'] = c19_cases['Summe Aachen'] / Einwohner_StadtAachen * 100000
c19_cases['Neuinfektionen 7T/100000 Aachen'] = c19_cases['Neuinfektionen Aachen'].rolling(7).sum() / Einwohner_StadtAachen * 100000
c19_cases['Neuinfektionen 7T/100000'] = c19_cases['Neuinfektionen'].rolling(7).sum() / Einwohner_StädteRegion * 100000


# Zwischenergebnis in Excel-Datei speichern

In [9]:
c19_cases.to_excel(config['Kennzahlen']['FileName'], 
                   sheet_name=config['Kennzahlen']['SheetName'],
                   index_label='Datum')